**Wiwino market analysis**

In [1]:
import sqlite3
import pandas as pd

connexion = sqlite3.connect("DB/vivino.db")
cursor = connexion.cursor()

We want to highlight 10 wines to increase our sales. Which ones should we choose and why?

Normalize the average rating to a common scale, usually between 0 and 1 (ratings_average/max rating value) 
Multiply the normalized average rating by the number of ratings. This gives more weight to products with both high ratings and a large number of ratings.
Sort the products based on their weighted score, with higher scores indicating better products.

In [25]:
top_ten_wines = """SELECT 
                name as wine_name,
                ROUND((ratings_average / 5) * ratings_count, 1) AS weighted_score,
                ratings_average,
                ratings_count
            FROM 
                wines 
            ORDER BY 
                weighted_score DESC
            LIMIT 10;
    """
top_ten_wines_df = pd.read_sql_query(top_ten_wines,connexion)
top_ten_wines_df

,wine_name,weighted_score,ratings_average,ratings_count
0,Cabernet Sauvignon,145308.5,4.6,157944
1,Brut Champagne,134666.8,4.6,146377
2,Tignanello,128261.7,4.5,142513
3,Sassicaia,99034.3,4.6,107646
4,60 Sessantanni Old Vines Primitivo di Manduria,84860.1,4.5,94289
5,Costasera Amarone della Valpolicella Classico,80321.4,4.3,93397
6,ARTEMIS Cabernet Sauvignon,71588.9,4.4,81351
7,Opus One,70888.8,4.6,77053
8,Pauillac (Premier Grand Cru Classé),67725.8,4.6,73615
9,Valbuena 5º,66961.3,4.6,72784


We have a limited marketing budget for this year. Which country should we prioritise and why?

Normalize each data point to a common scale between 0 and 1 to ensure equal weighting. 
Assign weights to each normalized data point based on its importance
Multiply each normalized data point by its respective weight and sum the results to get a priority score for each country
Rank countries based on their priority scores, with higher scores indicating higher priority

In [26]:
country_prioritise = """WITH max_values AS (
                    SELECT 
                        MAX(users_count) AS max_users,
                        MAX(regions_count) AS max_regions,
                        MAX(wines_count) AS max_wines,
                        MAX(wineries_count) AS max_wineries 
                    FROM 
                        countries
                )
                SELECT 
                    name as country_name,
                    ((regions_count / max_regions) * 0.1) + 
                    ((users_count / max_users) * 0.4) +
                    ((wines_count / max_wines) * 0.3) +
                    ((wineries_count / max_wineries) * 0.2) AS priority_score
                FROM 
                    countries, 
                    max_values 
                ORDER BY 
                    priority_score DESC;"""

country_prioritise_df = pd.read_sql_query(country_prioritise,connexion)
country_prioritise_df

,country_name,priority_score
0,France,0.6
1,États-Unis,0.4
2,Italie,0.0
3,Espagne,0.0
4,Portugal,0.0
5,Australie,0.0
6,Hongrie,0.0
7,Afrique du Sud,0.0
8,Allemagne,0.0
9,Chili,0.0


We would like to give awards to the best wineries. Come up with 3 relevant ones. Which wineries should we choose and why?

There are only 

We detected that a big cluster of customers likes a specific combination of tastes. We identified a few keywords that match these tastes: coffee, toast, green apple, cream, and citrus. We would like you to find all the wines that are related to these keywords. Check that at least 10 users confirm those keywords, to ensure the accuracy of the selection. Additionally, identify an appropriate group name for this cluster.

Selected 5 tastes from Keywords table and make sure more then 10 users  confirm by checking the count in keyword_wine table
Joined keywords, keywords_wine and wines table to get all the list of wine IDs & Names with atleast one taste mentioned above.

In [5]:
cluster_of_customers = """SELECT
                    wines.id AS wine_id,
                    wines.name AS wine_name,
                    COUNT(DISTINCT(keywords.name)) AS tastes_count,
                    keywords_wine.count AS number_of_users_confirm_keywords,
                    GROUP_CONCAT (DISTINCT(keywords.name)) AS list_of_tastes
                FROM
                    keywords
                JOIN
                    keywords_wine ON keywords.id = keywords_wine.keyword_id
                JOIN
                    wines ON keywords_wine.wine_id = wines.id
                WHERE
                    LOWER(keywords.name) IN ('coffee', 'toast', 'green apple', 'cream', 'citrus')
                    AND keywords_wine.count > 10
                GROUP BY
                    wines.id,
                    wines.name
                ORDER BY
                    tastes_count DESC;"""

cluster_of_customers_df = pd.read_sql_query(cluster_of_customers,connexion)
cluster_of_customers_df

,wine_id,wine_name,tastes_count,number_of_users_confirm_keywords,list_of_tastes
0,18931,La Grande Année Brut Champagne,5,525,"toast,coffee,cream,green apple,citrus"
1,74304,Cristal Brut Champagne (Millésimé),5,464,"toast,coffee,green apple,citrus,cream"
2,79162,Belle Epoque Brut Champagne,5,224,"green apple,toast,coffee,citrus,cream"
3,79631,Vintage,5,356,"toast,coffee,cream,green apple,citrus"
4,79836,La Grande Dame Brut Champagne,5,244,"toast,coffee,green apple,cream,citrus"
...,...,...,...,...,...
424,6255066,VV 26 Blanc de Blancs Vieilles Vignes Grand Cr...,1,11,toast
425,6343385,80 Vecchie Vigne Primitivo di Manduria Old Vines,1,12,coffee
426,6492988,Rioja Blanco Reserva,1,14,cream
427,6741558,Vigne Vecchie Gold Series Leggenda Primitivo d...,1,17,coffee


Joined keywords, keywords_wine and wines table to get all the list of wine IDs & Names with all taste mentioned above.

In [6]:
list_of_tastes_wines = """SELECT
                    wines.id AS wine_id,
                    wines.name AS wine_name,
                    COUNT(DISTINCT(keywords.name)) AS tastes_count,
                    keywords_wine.count AS number_of_users_confirm_keywords,
                    GROUP_CONCAT (DISTINCT(keywords.name)) AS list_of_tastes
                FROM
                    keywords
                JOIN
                    keywords_wine ON keywords.id = keywords_wine.keyword_id
                JOIN
                    wines ON keywords_wine.wine_id = wines.id
                WHERE
                    LOWER(keywords.name) IN ('coffee', 'toast', 'green apple', 'cream', 'citrus')
                    AND keywords_wine.count > 10
                GROUP BY
                    wines.id,
                    wines.name
				HAVING 
					tastes_count = 5;"""

list_of_tastes_wines_df = pd.read_sql_query(list_of_tastes_wines,connexion)
list_of_tastes_wines_df

,wine_id,wine_name,tastes_count,number_of_users_confirm_keywords,list_of_tastes
0,18931,La Grande Année Brut Champagne,5,525,"toast,coffee,cream,green apple,citrus"
1,74304,Cristal Brut Champagne (Millésimé),5,464,"toast,coffee,green apple,citrus,cream"
2,79162,Belle Epoque Brut Champagne,5,224,"green apple,toast,coffee,citrus,cream"
3,79631,Vintage,5,356,"toast,coffee,cream,green apple,citrus"
4,79836,La Grande Dame Brut Champagne,5,244,"toast,coffee,green apple,cream,citrus"
5,86684,Brut Champagne,5,2165,"toast,coffee,cream,green apple,citrus"
6,1105696,Trebbiano d'Abruzzo,5,21,"coffee,green apple,citrus,toast,cream"
7,1127349,Le Mesnil Blanc de Blancs (Cuvée S) Brut Champ...,5,240,"citrus,toast,coffee,cream,green apple"
8,1136950,Comtes de Champagne Blanc de Blancs,5,587,"toast,coffee,cream,citrus,green apple"
9,1153863,Sauternes,5,13,"green apple,coffee,toast,citrus,cream"


Find the top 3 most common grapes all over the world

In [7]:
most_common_grapes = """SELECT
                    COUNT(country_code) AS number_of_countries_using_grape,
                    GROUP_CONCAT(country_code) AS country_codes,
                    grape_id,
                    name AS grape_name
                FROM
                    most_used_grapes_per_country
                JOIN
                    grapes ON most_used_grapes_per_country.grape_id = grapes.id
                GROUP BY
                    grape_id
                ORDER BY
                    number_of_countries_using_grape DESC
                LIMIT 3;"""
most_common_grapes_df = pd.read_sql_query(most_common_grapes,connexion)
most_common_grapes_df

,number_of_countries_using_grape,country_codes,grape_id,grape_name
0,12,"us,es,au,hu,za,cl,md,ar,gr,il,ro,hr",2,Cabernet Sauvignon
1,11,"it,fr,hu,za,cl,md,gr,il,ro,hr,ch",10,Merlot
2,6,"it,fr,us,au,md,ar",5,Chardonnay


Find the 5 top rated wine from wines table where grape name is Cabernet Sauvignon which one of the top 3 used grape across world

In [10]:
top_wines_with_best_grape = """SELECT
                    wines.name,
                    ROUND((ratings_average / 5) * ratings_count, 1) AS weighted_score,
                    ratings_average,
                    ratings_count
                FROM
                    wines 
                where 
                    name like '%Cabernet Sauvignon%'
                ORDER BY 
                    weighted_score DESC
                LIMIT 5;"""

top_wines_with_best_grape_df = pd.read_sql_query(top_wines_with_best_grape,connexion)
top_wines_with_best_grape_df

,name,weighted_score,ratings_average,ratings_count
0,Cabernet Sauvignon,145308.5,4.6,157944
1,ARTEMIS Cabernet Sauvignon,71588.9,4.4,81351
2,Alexander Valley Cabernet Sauvignon,40538.1,4.4,46066
3,Special Selection Cabernet Sauvignon,38761.8,4.7,41236
4,Cabernet Sauvignon,25118.7,4.4,28544


Find the 5 top rated wine from wines table where grape name is Merlot which one of the top 3 used grape across world

In [11]:
top_wines_with_top_grapes ="""WITH grapes_list AS (
                        SELECT
                            COUNT(country_code) AS number_of_countries_using_grape,
                            GROUP_CONCAT(country_code) AS country_codes,
                            grape_id,
                            grapes.name AS grape_name
                        FROM
                            most_used_grapes_per_country
                        JOIN
                            grapes ON most_used_grapes_per_country.grape_id = grapes.id
                        GROUP BY
                            grape_id
                        ORDER BY
                            number_of_countries_using_grape DESC
                        LIMIT 3
                    )
                    SELECT *
                    FROM (SELECT
                        wines.name as wine_name,
                        ROUND((ratings_average / 5) * ratings_count, 1) AS weighted_score,
                        ratings_average,
                        ratings_count,
                        grapes_list.grape_name,
                        ROW_NUMBER() OVER (PARTITION BY CASE 
                                                                WHEN name LIKE  '%' || grapes_list.grape_name || '%' THEN grapes_list.grape_name
                                                            END 
                                                ORDER BY (SELECT NULL)) AS row_num
                    FROM
                        wines , grapes_list
                    where 
                        name like '%' || grapes_list.grape_name || '%'  
                    ORDER BY 
                        weighted_score DESC
                    ) AS subquery
                    WHERE row_num <= 5;"""

top_wines_with_top_grapes_df = pd.read_sql_query(top_wines_with_top_grapes,connexion)
top_wines_with_top_grapes_df

,name,weighted_score,ratings_average,ratings_count,grape_name,row_num
0,Cabernet Sauvignon,145308.5,4.6,157944,Cabernet Sauvignon,5
1,Cabernet Sauvignon,25118.7,4.4,28544,Cabernet Sauvignon,3
2,Chardonnay,16076.8,4.3,18694,Chardonnay,1
3,Cabernet Sauvignon (Signature),12631.6,4.6,13730,Cabernet Sauvignon,1
4,Chateau Buena Vista Cabernet Sauvignon Napa Va...,11536.9,4.3,13415,Cabernet Sauvignon,2
5,Cabernet Sauvignon,2119.0,4.4,2408,Cabernet Sauvignon,4
6,Merlot (Signature),1988.3,4.3,2312,Merlot,1
7,Merlot,1691.4,4.4,1922,Merlot,3
8,One Sixteen Chardonnay,1542.6,4.4,1753,Chardonnay,5
9,Calle Merlot Rosso Veronese,926.6,4.4,1053,Merlot,2


A country leaderboard. A visual that shows the average wine rating for each country

In [12]:
country_leaderboard = """SELECT 
                    countries.name AS country_name,
                    ROUND(AVG(ratings_average), 2) AS average_ratings
                FROM 
                    countries
                JOIN 
                    regions ON countries.code = regions.country_code
                JOIN 
                    wines ON wines.region_id = regions.id
                GROUP BY 
                    country_name
                ORDER BY 
                    average_ratings DESC;"""

country_leaderboard_df = pd.read_sql_query(country_leaderboard,connexion)
country_leaderboard_df

,country_name,average_ratings
0,Israël,4.50
1,Allemagne,4.50
2,États-Unis,4.49
3,Moldavie,4.48
4,Hongrie,4.47
5,Australie,4.46
6,Afrique du Sud,4.46
7,France,4.45
8,Portugal,4.44
9,Espagne,4.44


In [19]:
import plotly.express as px

fig = px.bar(country_leaderboard_df, x = "country_name", y = "average_ratings", title= "Country Leaderboard For Average Wine Ratings")
fig.show()

In [20]:
country_vintage_leaderboard = """SELECT 
                    countries.name AS country_name,
                    ROUND(AVG(vintages.ratings_average), 2) AS average_ratings
                FROM 
                    countries
                JOIN 
                    regions ON countries.code = regions.country_code
                JOIN 
                    wines ON wines.region_id = regions.id
				JOIN 
					vintages ON wines.id = vintages.wine_id
                GROUP BY 
                    country_name
                ORDER BY 
                    average_ratings DESC;"""

country_vintage_leaderboard_df = pd.read_sql_query(country_vintage_leaderboard,connexion)
country_vintage_leaderboard_df

,country_name,average_ratings
0,Roumanie,4.50
1,Croatie,4.40
2,Argentine,4.32
3,Chili,4.12
4,Portugal,3.93
5,Moldavie,3.87
6,États-Unis,3.70
7,Afrique du Sud,3.66
8,Italie,3.63
9,Espagne,3.38


In [21]:
fig = px.bar(country_vintage_leaderboard_df, x = "country_name", y = "average_ratings", title= "Country Leaderboard For Average Vintage Ratings")
fig.show()

A vintage leaderboard. Average wine rating for all top rating vintages

In [14]:
vintage_leaderboard = """SELECT 
                    vintages.name AS vintage_name,
                    ROUND(AVG(wines.ratings_average), 2) AS wines_average_ratings,
                    wine_id
                FROM 
                    vintages
                JOIN 
                    wines ON wines.id = vintages.wine_id
                JOIN 
                    vintage_toplists_rankings ON vintages.id = vintage_toplists_rankings.vintage_id
                GROUP BY 
                    wine_id
                ORDER BY 
                    wines_average_ratings DESC;"""

vintage_leaderboard_df = pd.read_sql_query(vintage_leaderboard,connexion)
vintage_leaderboard_df

,vintage_name,wines_average_ratings,wine_id
0,Scarecrow Cabernet Sauvignon 2015,4.8,1611255
1,Masseto Toscana 2006,4.7,2446729
2,Quintarelli Giuseppe Veneto Alzero Cabernet 2007,4.7,1194532
3,Salon Le Mesnil Blanc de Blancs (Cuvée S) Brut...,4.7,1127349
4,Dal Forno Romano Amarone della Valpolicella Mo...,4.7,89750
...,...,...,...
99,Tommasi Amarone della Valpolicella Classico 2018,4.3,7103
100,Buena Vista Chateau Buena Vista Cabernet Sauvi...,4.3,5261
101,Chateau Montelena Chardonnay 2019,4.3,1697
102,Château Gazin Pomerol 2009,4.2,18555


One of our VIP clients likes Cabernet Sauvignon and would like our top 5 recommendations.

In [16]:
recommendations_for_vip_clients = """SELECT
                    wines.name AS wine_name,
                    grapes.name AS grape_name,
                    wines.ratings_average AS average_rating,
                    wines.ratings_count AS rating_count,
                    ROUND((wines.ratings_average / 5) * wines.ratings_count, 1) AS weighted_score
                FROM
                    wines
                JOIN
                    regions ON regions.id = wines.region_id
                JOIN
                    most_used_grapes_per_country ON most_used_grapes_per_country.country_code = regions.country_code
                JOIN
                    grapes ON grapes.id = most_used_grapes_per_country.grape_id
                WHERE
                    grape_name = 'Cabernet Sauvignon'
                ORDER BY
                    weighted_score DESC
                LIMIT 5;"""

recommendations_for_vip_clients_df = pd.read_sql_query(recommendations_for_vip_clients,connexion)
recommendations_for_vip_clients_df

,wine_name,grape_name,average_rating,rating_count,weighted_score
0,Cabernet Sauvignon,Cabernet Sauvignon,4.6,157944,145308.5
1,ARTEMIS Cabernet Sauvignon,Cabernet Sauvignon,4.4,81351,71588.9
2,Opus One,Cabernet Sauvignon,4.6,77053,70888.8
3,Valbuena 5º,Cabernet Sauvignon,4.6,72784,66961.3
4,Ribera del Duero,Cabernet Sauvignon,4.5,69278,62350.2


In [22]:
fig = px.bar(recommendations_for_vip_clients_df, x = "wine_name", y = "weighted_score", title= "Top 5 recommendations of wine with weighted score")
fig.show()